<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/%E7%9F%A5%E4%B9%8E%E7%AC%94%E8%AE%B0%EF%BC%9AELMO%E8%AF%A6%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.前言

&emsp;&emsp;给定N个tokens的序列$(t_1,t_2,\ldots,t_k)$，前向LM就是给定前k-1个输入序列$(t_1,t_2,\ldots,t_{k-1})$的hidden state预测第$k$个位置的token;反向LM就是给定后面的序列预测之前的token。然后将LM的第k个位置的hidden state 输出作为word embedding。这种做法的缺点是对于每一个单词都有唯一的固定的embedding表示，而对于多义词，这种做法显然不对。而ELMO的做法是先使用上述做法训练出单词的word embedding，然后在给定的上下文中动态的修改该word embedding，这样单词就是上下文相关的了，很大程度上缓解了歧义的发生。

## 2. 双向语言模型

&emsp;&emsp;ELMO用到上文提到的双向LM,给定N个tokens$t_1,t_2,\ldots,t_N$，LM通过给定前面的k-1个位置的token序列计算第k个token出现的概率：
$$
p(t_1,t_2,\ldots,t_N)=\prod _{k=1}^Np(t_k|t_1,t_2,\ldots,t_{k-1})
$$
反向LM的计算类似：
$$
p(t_1,t_2,\ldots,t_N)=\prod _{k=1}^Np(t_k|t_{k+1},t_{k+2},\ldots,t_{N})
$$

ELMO使用BiLSTM进行训练，训练目标是最大化：
$$
\sum _{k=1}^N(logp(t_k|t1,\ldots,t_{k-1};\theta _x,\overrightarrow{\theta}_{LSTM},\theta _s)+logp(t_k|t_{k+1},\ldots,t_N;\theta _x,\overleftarrow{\theta}_{LSTM},\theta _s))
$$

对于每个token $t_k$，通过一个L层的BiLSTM计算出2L+1个表示：

$$
R_k=\{ x_k^{LM},\overrightarrow{h}_{k,j}^{LM},\overleftarrow{h}_{k,j}^{LM}|j=1,\ldots,L \}=\{h_{kj}^{LM}|j=0,\ldots,L\}
$$
其中$x_k^{LM}$是对token进行普通的embedding的结果，$h_{k0}^{LM}$代表$x_k^{LM}$，$h_{kj}^{LM}=[\overrightarrow{h}_{k,j}^{LM};\overleftarrow{h}_{k,j}^{LM}]$是每个BiLSTM层的输出结果。最上面一层的$h_{kj}^{LM}$是用softmax来预测下面一个单词$t_{k+1}$。\

应用中将ELMO中所有层的输出R压缩为单个向量,通用的做法如下：
<img src="https://www.zhihu.com/equation?tex=E+L+M+o_%7Bk%7D%5E%7Bt+a+s+k%7D%3DE%5Cleft%28R_%7Bk%7D+%3B+%5CTheta%5E%7Bt+a+s+k%7D%5Cright%29%3D%5Cgamma%5E%7Bt+a+s+k%7D+%5Csum_%7Bj%3D0%7D%5E%7BL%7D+s_%7Bj%7D%5E%7Bt+a+s+k%7D+h_%7Bk%2C+j%7D%5E%7BL+M%7D" width="500" />

其中$s^{task}$是一个softmax出来的结果，$\gamma$是一个任务相关的scale参数。$\gamma$在不同任务中取不同的值效果会有较大的差异。

## 3. ELMO网络结构及预训练

<img src="https://pic2.zhimg.com/80/v2-945ea2c964e548cb9a9128864b5f6d49_720w.jpg" width="500" />

* 将输入转换为普通的word embedding
* 将word embedding作为一个两层的BiLSTM的输入

本质上，事先使用语言模型学好单词的Word Embedding ，此时多义词无法区分。然后根据上下文单词的语义去调整单词的word embedding 表示，这样经过调整后的Word embedding  更能表达在这个上下文中的具体含义，也可以解决一词多义的问题。所以ELMO本身是根据当前上下文对Word Embedding动态调整的思路。\
原始论文中是通过CNN对字符级进行编码以获取上下文无关的word embedding，然后使用两层的BiLSTM进一步训练，并将三层的输出scale到1024维，最后对每个token输出3个1024维的向量表示。\

使用这个网络结构利用大量语料做语言模型任务就能预先训练好这个网络，如果训练好这个网络后，输入一个新句子Snew，句子中每个单词都能得到对应的三个Embedding:最底层是单词的 Word Embedding，往上走是第一层双向LSTM中对应单词位置的 Embedding，这层编码单词的句法信息更多一些；再往上走是第二层LSTM中对应单词位置的 Embedding，这层编码单词的语义信息更多一些。也就是说，ELMO 的预训练过程不仅仅学会单词的 Word Embedding，还学会了一个双层双向的LSTM网络结构，而这两者后面都有用

## 4. ELMO预训练模型使用


<img src="https://pic2.zhimg.com/80/v2-e53a17ada1d510958215cc860c33efb9_720w.jpg" width="500" />

预训练好网络结构后，如何给下游任务使用呢？上图展示了下游任务的使用过程，比如我们的下游任务仍然是 QA 问题，此时对于问句 X，我们可以先将句子 X 作为预训练好的 ELMO 网络的输入，这样句子 X 中每个单词在 ELMO 网络中都能获得对应的三个 Embedding，之后给予这三个 Embedding 中的每一个 Embedding 一个权重a，这个权重可以学习得来，根据各自权重累加求和，将三个 Embedding 整合成一个。

然后将整合后的这个 Embedding 作为 X 句在自己任务的那个网络结构中对应单词的输入，以此作为补充的新特征给下游任务使用。对于上图所示下游任务 QA 中的回答句子 Y 来说也是如此处理。

因为 ELMO给下游提供的是每个单词的特征形式，所以这一类预训练的方法被称为“Feature-based Pre-Training”。


## 5. 优缺点

###5.1 优点

* ELMO引入上下文动态调整单词的embedding能够解决一词多义的问题。

* 适用范围广，普适性强，在分类、阅读理解等多个任务都有不错的效果。

### 5.2 缺点

* 在特征抽取器的选择上选择BiLSTM没有使用Transformer。Transformer 提取特征的能力是要远强于LSTM的。

* ELMO 采取双向拼接这种融合特征的能力可能比 Bert 一体化的融合特征方式弱








